In [1]:
# -*- coding: utf-8 -*-

%matplotlib inline
import numpy as np

# Genetic Trading Algorithm (GTA)

The principle of the GTA is to exploit the genetic operators of breeding, mutating and selecting, to devise an algorithm to select a portfolio.

## Strategy of an individual

Given a market with $m$ stocks, we will devise a strategy that we will represent by an individual of the population.
The sequence characterizing an individual will consist of 3 parts :

### 1 - The Strategy binaries

As for the Genetic Algorithm for Rock, Paper, Scissors, we will suppose that the $2^m$ first binaries will be used as a strategy for a given market :

For instance suppose $m=3$, and we have GOOG, SPY and YHOO as stocks in our market. <br>
We will denote an increase in the stock relative price by a $1$, and a decrease by a $0$.

We can thus represent every possible market allocation of $m$ stocks with $2^m$ :<br>
Here with $m=3$, $000$ means that GOOG, SPY and YHOO went down, $001$ means GOOG and SPY went down, YHOO up, etc ...

$\forall i \in [[0,2^m-1]]$, we will store in the $i$-th position, the action the individual in the $i$-th market situation. <br>
We code this action as a $m$ dimensional array of binaries that we denote $A$, and for $\forall j \in [[1,m]]$, if $A(j)=1$ then we buy stock j, if $A(j) = 0$ then we sell stock j.

To sum up, one of the table of an individual may look like this for the previous case with $m=3$ :<br>
In the $000$-th index of the individual table, we have $101$ which translates in "If GOOG, SPY and YHOO went down, then buy GOOG, sell SPY, and buy YHOO".

### 2 - The aggresivity factor

We will code in 3 binaries the aggresivity factor, i.e. how much will the individual change his portfolio.
For instance suppose, that for a given market allocation, he decides to buy GOOG and sell YHOO, how much will he buy or sell ? 

Well one way to do this is to define this as an increase or decrease of the quantity of the given stock by a certain percentage. We denote by $\zeta$ the decimal value of the number coded in the 3 binaries.

We give two possible ways :
<ol>
<li>Randomness : we pick a random number $x$ between $0$ and $\zeta$ and we will increase/decrease the number of stock by $x$.</li>
<li>Brute : we simply increase/decrease the % of stock by $\zeta$</li>
</ol>

__Remark__ : I chose 3 binaries so that we do not exceed change of 7%, as it is first costly to do such changes of position, and it will require a higher computational time.

### The defensive factor

Given the market volatility, it may be safer to hold your position in certain cases. <br>
We translate it by a defensive factor that we will code in a few binaries to reflect our incertainty about the market.

We give ourselves a safety threshold common to all individuals $\alpha$. We also denote by $\xi$ the decimal value of the number coded in those binaries (_number of binaries yet to decide_).We can approach this problem in two ways :
<ol>
<li> Safety in action : we define a _caprice tendency_ :
$$\mathbb{P}(caprice_i) = \alpha * \xi * mistakes$$
with $\mathbb{P}(caprice_i)$ the probability for a given stock $i$ that when the individual has to increase or decrease his position of a given stock, he decides to do nothing. $mistakes$ for a givne stock is the consecutive number of time the individual failed to predict whether it would increase or decrease.</li>
<li> Variance safety : if historically the STD of the given stock surpasses a threshold, we choose not to deal with this stock : 
$$ STD(stock_i) > \alpha * \xi \Longrightarrow \text{We do nothing}$$
</li>
</ol>
__Remark__ : I think we should code $\xi$ on 3 or binaries.

With all this we can define an individual, with all its genes coded in binaries so the genetic operations will be easy to apply.

## Translation to a portfolio

The first day we will suppose we will play the uniform portfolio $b = (1/m,...,1/m)$, and after the first day we will inject the strategy of the fittest individual : based on the market vector, we will receive a $m$ dim array with binaries which will enable us to transfer money based on this suggestion :

We increase each stock by a given percentage, and if ever it is not on the convex set we project it (__to review I think__).

For a given day $t$, we denote by $\tilde{b(t)}$ the portfolio suggested by the algorithm, and $b(t)$ the final portfolio we will use for this day.
I propose that at time $t+1$, we always compute the portfolio $\tilde{b(t+1)}$ and depending on the performances of $b(t)$ we define $b(t+1)$ as a combinaison of those two :

$$b(t+1) = \gamma b(t) + (1 - \gamma) \tilde{b(t+1)}$$

with \gamma a dynamic parameter based on the performances of $b(t)$.

# Probable downsides

There are several issues that can be already seen from this model :

<ol>
<li>__Computational cost__ : exponential size of the sequence of each individual, which makes the calculation heavy. Not a problem if we trade daily, but for in day trading it may be not be good enough compared to other algorithms.

__Remark__ : One way to solve this problem may be to reduce $m$, and choose a subset $n \leq m$ of stocks to trade, based on different criteria :
    <ol>
    <li>Their Sharpe ratio : whether those stacks have a good return / variance</li>
    <li>How correlated those actions are : we may want to pick the most uncorrelated or correlated stocks depending on our vision of the market.</li>
    </ol>
<li>__Trading commission__ : trading commission have to be taken into account, and we may want to limit the number of trades. We also trade with closing prices, if we want to devise in-day trading strategies we may have to pay to have access to in-day trading prices</li>
</ol>